# Parallel: get_patch_location_array *(both include and reject arrays)*
    * Central to the functionality of all use cases.
    * Needs flexibility for different select methods.
    * May be the slowest part of the whole algorithm.
    
## Concept: separate the location-selection algorithm from the output method
    * Allows paralellization of the slowest part - include | reject patch location.
    * Decouple from the get-write output (dir, tfrecord, ...).
    * Allow inspection / select-method parameter modification before making output.
    * Insertion point for new or modified location-selection methods.
    
****
## Selection Algorithm Central
```python
#                   common to both
grey_thumbnail = np.array(thumbnail.convert("L"))
thresh = threshold_otsu(grey_thumbnail)
mask = np.array(grey_thumbnail) < thresh
# mask_im = PIL.Image.fromarray(np.uint8(mask_im) * 255)

#                   new in opneslide_2_tfrecord.py
thresh = 80
np_img = np.array(one_thumb.convert('RGB'))
np_img = rgb2lab(np_img)
np_img = np_img[:,:,0]
mask_im = np.array(np_img) < thresh
# mask_im = PIL.Image.fromarray(np.uint8(mask_im) * 255)
```

```python
"""
                    87: wsi-sampler/sampler.py
"""

if np.sum(mask[x:x_mask_window, y:y_mask_window]) > 0:
    # include this location ...
```
```python
"""
                    504-505: DigiPath_MLTK/src/python/openslide_2_tfrecord.py
"""

mask_value = np.float(np.sum(thumb_arr==0)) / np.float(np.prod(thumb_arr.shape))
if mask_value <= drop_threshold:
    # include this location ...
    
```
****
### Proto function to sleep on
```python
def get_patch_location_array(run_parameters):
    
    select_list = []
    reject_list = []
    """
        Create temporary dirs
        
    """
    # common for parallel: temp_dir_reject_locations, temp_dir_select_locations
    temp_dir_reject_locations = TemporaryDirectory()
    temp_dir_select_locations = TemporaryDirectory()
    def threshold_otsu_selector():
        # get the mask
        # get the rows & cols array
        # parallelize on selection-worker
        # ...
        
    def threshold_rgb2lab_selector():
        # get the mask
        # get the rows & cols array
        # parallelize on selection-worker,
        # ...
        
    SELECT_LOCATION_METHOD = {'threshold_otsu', threshold_otsu_selector, 
                              'threshold_rgb2lab': threshold_rgb2lab_selector}
    
    patch_select_method = run_parameters['patch_select_method']
    
    if patch_select_method in SELECT_LOCATION_METHOD:
        rc = SELECT_LOCATION_METHOD[patch_select_method](run_parameters)
    
    # collect
    if rc == 0:
        # collect the mini-files into the select and reject location lists
        select_list = list_from_temp_dir(temp_dir_select_locations)
        reject_list = list_from_temp_dir(temp_dir_reject_locations)
        
    # cleanup
    temp_dir_reject_locations.cleanup()
    temp_dir_select_locations.cleanup()
    
    return select_list, reject_list
        
```